# Image Augmentation and Preprocessing Utilities
This notebook provides utility functions for image preprocessing and augmentation commonly used in deep learning tasks.

#

In [ ]:
# Import necessary libraries
import os
import numpy as np
from PIL import Image
import albumentations as A
import matplotlib.pyplot as plt

#%

## Image Loading and Preprocessing

#

In [ ]:
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    """
    Load and preprocess an image.

    Args:
        image_path (str): Path to the image file
        target_size (tuple): Desired output size (width, height)

    Returns:
        PIL.Image: Preprocessed image
    """
    # Load image
    img = Image.open(image_path)

    # Convert to RGB if necessary
    if img.mode != 'RGB':
        img = img.convert('RGB')

    # Resize image
    img = img.resize(target_size, Image.Resampling.LANCZOS)

    return img

def load_images_from_directory(directory_path, target_size=(224, 224)):
    """
    Load all images from a directory and preprocess them.

    Args:
        directory_path (str): Path to the directory containing images
        target_size (tuple): Desired output size (width, height)

    Returns:
        list: List of preprocessed images
    """
    images = []
    valid_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}

    for filename in os.listdir(directory_path):
        if os.path.splitext(filename)[1].lower() in valid_extensions:
            image_path = os.path.join(directory_path, filename)
            try:
                img = load_and_preprocess_image(image_path, target_size)
                images.append(img)
            except Exception as e:
                print(f"Error loading {filename}: {str(e)}")

    return images

#%

## Image Augmentation Functions

In [ ]:
class BasicAugmenter:
    def __init__(self, p=0.5):
        """
        Initialize the augmentation pipeline.

        Args:
            p (float): Probability of applying each augmentation
        """
        self.transform = A.Compose([
            A.HorizontalFlip(p=p),
            A.RandomRotate90(p=p),
            A.VerticalFlip(p=p),
            A.RandomBrightnessContrast(p=p),
            A.RandomCrop(height=178, width=178, p=p),
            A.HueSaturationValue(p=p),
            A.GaussNoise(p=p)
        ])

    def augment(self, image):
        """
        Apply augmentations to an image.

        Args:
            image (PIL.Image): Input image

        Returns:
            PIL.Image: Augmented image
        """
        # Convert PIL image to numpy array
        image_np = np.array(image)

        # Apply augmentations
        augmented = self.transform(image=image_np)['image']

        # Convert back to PIL image
        return Image.fromarray(augmented)

## Example Usage

In [ ]:
def display_augmentations(image, n_examples=5):
    """
    Display original image and multiple augmented versions.

    Args:
        image (PIL.Image): Input image
        n_examples (int): Number of augmented examples to show
    """
    augmenter = BasicAugmenter(p=0.7)

    plt.figure(figsize=(15, 3))

    # Display original image
    plt.subplot(1, n_examples + 1, 1)
    plt.imshow(image)
    plt.title('Original')
    plt.axis('off')

    # Display augmented versions
    for i in range(n_examples):
        augmented_image = augmenter.augment(image)
        plt.subplot(1, n_examples + 1, i + 2)
        plt.imshow(augmented_image)
        plt.title(f'Augmented {i+1}')
        plt.axis('off')

    plt.tight_layout()
    plt.show()

#%

## Example: Load and augment a single image
To use this code, replace 'path_to_your_image.jpg' with the path to your image:

#

In [ ]:
# Example usage (commented out - replace with your image path)
"""
# Load and preprocess a single image
image = load_and_preprocess_image('path_to_your_image.jpg')

# Display original and augmented versions
display_augmentations(image)

# Load multiple images from a directory
images = load_images_from_directory('path_to_your_image_directory')
"""